# 機械学習

## 目次

- 項目の説明
- 欠損値の扱い
- カテゴリ変数の扱い

## Section2 データの解説

In [ ]:
# # Googleドライブのマウント（Colab使いのみ）

# from google.colab import drive
# drive.mount('/content/drive')

# %cd /content/drive/MyDrive/dlc/week1

In [ ]:
# 図表が使えるようにする

import matplotlib
%matplotlib inline
import seaborn as sns
sns.set()

In [ ]:
# データのロード

import pandas as pd

data = pd.read_csv("./data/train.csv")

### 2.1 項目の説明

In [ ]:
# 先頭3行を表示

data.head(3)

- PassengerId – 乗客識別ユニークID
- Survived – 生存フラグ（0=死亡、1=生存）　←　これを予測する
- Pclass – チケットクラス（1st, 2nd, 3rd）
- Name – 乗客の名前
- Sex – 性別（male=男性、female＝女性）
- Age – 年齢
- SibSp – タイタニックに同乗している兄弟/配偶者の数
- Parch – タイタニックに同乗している親/子供の数
- Ticket – チケット番号
- Fare – 料金
- Cabin – 客室番号
- Embarked – タイタニックへ乗った港（C=Cherbourg, S=Southampton, Q=Queenstown）

In [ ]:
# データのサイズを確認

data.shape

In [ ]:
# データの欠損・型を確認

data.info()

In [ ]:
# 欠損率を確認

def nullCountFig(df):
    null_val = df.isnull().sum()
    percent = 100 * df.isnull().sum()/len(df)
    counted_table = pd.concat([null_val, percent], axis=1)
    counted_figure = counted_table.rename(
        columns = {0 : '欠損数', 1 : '欠損率(%)'}
    )
    return counted_figure

nullCountFig(data)   

### 2.2 欠損値の扱い

欠損には大きく3種類あります。

- 完全にランダムに欠損
- 観測データに依存する欠損（特定のデータの中でランダムに欠損）
- 欠損データに依存する欠損（ランダムではない）

これらを考慮した上で、欠損値に対応する主な方針は4つです。

1. 欠損のある行、列を除外する
2. 何らかの値で埋める（単変量補完】
3. 何らかの値で埋める（多変量補完）
4. 欠損値を受け入れてくれるモデルを使う

In [ ]:
# 欠損のある行番号を取得

miss_index = data[data.isnull().any(axis=1)].index
miss_age_index = data[data['Age'].isnull()].index
miss_cabin_index = data[data['Cabin'].isnull()].index
miss_embarked_index = data[data['Embarked'].isnull()].index

#### 1. 欠損のある行、列を除外する

メリット：かんたん

デメリット：予測性能の低下を招きやすい

In [ ]:
# 特定のデータを除外する

non_null_data1 = data.drop(['Age', 'Cabin', 'Embarked'], axis=1)
display(non_null_data1.head(3))

In [ ]:
# 特定のデータを取り出す（上と同じ）

non_null_data2 = data[['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'SibSp', 'Parch', 'Ticket', 'Fare']]
display(non_null_data2.head(3))

In [ ]:
# 欠損値が一つでも含まれる列を除外する

non_null_data3 = data.dropna()
display(non_null_data3.head(5))

#### 2. 何らかの値で埋める（単変量補完）

メリット：かんたん

デメリット：欠損が多いと効果薄

In [ ]:
def highlight(df):
    styles = df.copy()
    styles.iloc[[miss_age_index], :] = 'background-color: red'
    return styles

In [ ]:
# 共通の値（0）で埋める

simple_filled_data1 = data.fillna(0)
display(simple_filled_data1.head(5))

In [ ]:
# 平均値で埋める

## 平均値：.mean()
## 中央値：.median()
## 最瀕値：.mode()
simple_filled_data2 = data.fillna(data.mean())
display(simple_filled_data2.head(5))

In [ ]:
# 直前or直後の値で埋める（時系列データ向き）

## 直前：method='ffill'
## 直後：method='bfill'
simple_filled_data3 = data.fillna(method = 'ffill')
display(simple_filled_data3.head(5))

#### 3. 何らかの値で埋める（多変量補完）

メリット：予測精度が高くなりがち

デメリット：うまくいかなかった時、原因がわからないがち

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import matplotlib.pyplot as plt
import numpy as np

# 数値だけのデータを作る
only_num_data = data.drop(['PassengerId', 'Survived', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], axis=1)
only_num_data_columns = only_num_data.columns

# ヒストグラムを作る関数
def figmake(df, key):
    x_bin = 50
    x_max = df[key].max()
    x_min = df[key].min()
    bins = np.linspace(x_min, x_max, x_bin)
    plt.figure()
    plt.hist(only_num_data[key], bins=bins, color='red', alpha=0.5)
    plt.hist(df[key], bins=bins, color='blue', alpha=0.5)
    plt.title(key)
    plt.ylabel('count')
    plt.show()

In [ ]:
# ベイジアンブリッジを使う

bayesian = pd.DataFrame(
                IterativeImputer().fit_transform(only_num_data)
                , columns=only_num_data_columns
            )

display(bayesian)
figmake(bayesian, 'Age')

In [ ]:
# ランダムフォレストを使う

from sklearn.ensemble import RandomForestRegressor

randf = pd.DataFrame(
            IterativeImputer(RandomForestRegressor()).fit_transform(only_num_data)
            , columns=only_num_data_columns
        )

display(randf)
figmake(randf, 'Age')

In [ ]:
# KNNを使う

from sklearn.impute import KNNImputer

knn = pd.DataFrame(
            KNNImputer(n_neighbors=2).fit_transform(only_num_data)
            , columns=only_num_data_columns
        )

display(knn)
figmake(knn, 'Age')

#### 4. 欠損値を受け入れてくれるモデルを使う

メリット：予測精度が高くなりがち

デメリット：ちょっとめんどくさい

XGBoost, LightGBMが有名

### 2.3 カテゴリ変数の扱い

文字列のままでは基本的にモデルへ入力することができないため、代わりとなる数値に変換する必要があります。

また、文字列データのみならず質的なデータは量的なデータに比べ扱いにくい場合があります。

そこで、[A,B,C]のような複数の値のある要素をAの有無という形式 **（ダミー変数 (Dummie variable)）** に変換することで扱いやすくします。この処理を **One-Hot エンコーディング (One hot encodring)** といいます。

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# 乗船港のみのデータ作成
embarked_data = data['Embarked'].dropna()

# エンコーダの定義
encoder = OneHotEncoder(sparse=False)

display(
    pd.DataFrame(
        encoder.fit_transform(embarked_data.values.reshape(-1, 1))
        ,columns=encoder.categories_
    )
)

### 2.4 特徴量エンジニアリング

**特徴量エンジニアリング (Feature engineering)** とは、機械学習​​モデル​の性能​を向上させるために、今あるデータの特徴量から新たな特徴量を構築することです。

In [ ]:
# コピーを作る

fe_data = data.copy()

In [ ]:
# 何人家族で乗船したかという特徴量を作る

fe_data['family_size'] = fe_data['SibSp'] + fe_data['Parch'] + 1

fig,ax = plt.subplots(1, 2, figsize=(16, 8))
# 生存者と死亡者の数を確認
sns.countplot(fe_data['family_size'],hue=data['Survived'], ax=ax[0])
# family_sizeごとの生存率を確認
sns.barplot(x='family_size', y='Survived', data=fe_data, color='orange', ax=ax[1])

In [ ]:
# 名前の敬称という特徴量を作る

fe_data['honorific'] = fe_data['Name'].map(lambda x: x.split(', ')[1].split('. ')[0])

print(fe_data['honorific'].value_counts())

In [ ]:
# 似ている敬称をまとめる

fe_data['honorific'].replace(['Capt', 'Col', 'Major', 'Dr', 'Rev'], 'Officer', inplace=True)
fe_data['honorific'].replace(['Don', 'Sir',  'the Countess', 'Lady', 'Dona', 'Jonkheer'], 'Royalty', inplace=True)
fe_data['honorific'].replace(['Mme', 'Ms'], 'Mrs', inplace=True)
fe_data['honorific'].replace(['Mlle'], 'Miss', inplace=True)

print(fe_data['honorific'].value_counts())

In [ ]:
# 敬称ごとの年齢を確認

# Ageの欠損がないデータを作成
fe_data_age = fe_data.dropna(subset=['Age']).copy()

fig, ax = plt.subplots()
sns.boxplot(x='honorific', y='Age', data=fe_data_age, color='coral')